This notebook performs the core of the post-processing procedure for crosstalk tomography: It estimates the local fidelities for all two-qubit Cliffords, from which the pinched marginal can then be reconstructed.

The reconstruction and plots are performed in the Python notebook "crosstalk_tomography_python.ipynb".

The results of this notebook are already provided in the "data" subfolder, so that **it does not need to be run**. Re-running this notebook will take a significant amount of time.

In [1]:
# this loads all the Julia functions and also some of the Python functions via PyCall
include("julia_functions.jl");

# directly import Python modules:
np = pyimport("numpy");
qi = pyimport("qiskit.quantum_info");

In [2]:
# import PTMs 
const ptms = load_pickle("application2/ptms");

# set parameters
lengths = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
number_qubits = 2
number_samples = 15000 

# unfortunately, the pickle-import flattens Python's list of lists structure. This reshapes it back to the expected index behavior: 
ptms_reshaped = Array{Vector{Matrix{Float64}}}(undef, size(lengths)[1], number_samples);

for i in 1:size(lengths)[1]
    for j in 1:number_samples
        
        gates_array = Array{Matrix{Float64}}(undef, lengths[i])
        
        for k in 1:lengths[i]
            gates_array[k] = ptms[i][j + number_samples*(k-1)]
        end
        ptms_reshaped[i, j] = gates_array
    end
end

In [3]:
# vectorized initial state and POVMs:
initial_state = get_initial_state_vec();
measurements = get_measurement_vecs();

# create all two-qubit Cliffords:
two_qubit_cliffords = get_two_qubit_cliffords();

# projection matrices for the two-qubit local Clifford protocol:
projection_matrices = get_projection_matrices();
proj01 = projection_matrices[2];
proj10 = projection_matrices[3];
proj11 = projection_matrices[4];

# import probs and replace NaN by zeros:
probs_xx = npzread("noisy_data_xx_crosstalk.npy");
probs_zz = npzread("noisy_data_zz_crosstalk.npy")

replace_nan(probs_xx);
replace_nan(probs_zz);

In [15]:
# estimate local fidelities for all two-qubit Cliffords, for data including XX crosstalk:

xx01 = zeros(11520)
xx10 = zeros(11520)
xx11 = zeros(11520)

for i in 1:11520
    
    A = np.real(qi.PTM(qi.Operator(two_qubit_cliffords[i])));
    
    A_projected = proj11 * A * proj11;
    
    # 01:
    k_values01 = get_correlators_views(ptms_reshaped, probs_xx, A, 2, 15000, 3000, 5, lengths, "local01", initial_state, measurements, false);

    decay_rate01 = get_decay_rate(k_values01, lengths);
  
    xx01[i] = decay_rate01;
    
    # no crosstalk, 10:
    k_values10 = get_correlators_views(ptms_reshaped, probs_xx, A, 2, 15000, 3000, 5, lengths, "local10", initial_state, measurements, false);

    decay_rate10 = get_decay_rate(k_values10, lengths);
  
    xx10[i] = decay_rate10;
    
    # no crosstalk, 11:
    k_values11 = get_correlators(ptms_reshaped, probs_xx, A_projected, 2, 15000, 3000, 5, lengths, "local11", proj11, initial_state, measurements, false);

    decay_rate11 = get_decay_rate(k_values11, lengths);
  
    xx11[i] = decay_rate11;

end


# save the Julia vectors of decay rates as npz files, so they can be imported to Python:
npzwrite("data/xx01.npz", xx01)
npzwrite("data/xx10.npz", xx10)
npzwrite("data/xx11.npz", xx11)

In [20]:
# repeat for ZZ crosstalk:

zz01 = zeros(11520)
zz10 = zeros(11520)
zz11 = zeros(11520)

for i in 1:11520
    
    A = np.real(qi.PTM(qi.Operator(two_qubit_cliffords[i])));
    
    A_projected = proj11 * A * proj11;
    
    # 01:
    k_values01 = get_correlators_views(ptms_reshaped, probs_zz, A, 2, 15000, 3000, 5, lengths, "local01", initial_state, measurements, false);

    decay_rate01 = get_decay_rate(k_values01, lengths);
  
    zz01[i] = decay_rate01;
    
    # no crosstalk, 10:
    k_values10 = get_correlators_views(ptms_reshaped, probs_zz, A, 2, 15000, 3000, 5, lengths, "local10", initial_state, measurements, false);

    decay_rate10 = get_decay_rate(k_values10, lengths);
  
    zz10[i] = decay_rate10;
    
    # no crosstalk, 11:
    k_values11 = get_correlators(ptms_reshaped, probs_zz, A_projected, 2, 15000, 3000, 5, lengths, "local11", proj11, initial_state, measurements, false);

    decay_rate11 = get_decay_rate(k_values11, lengths);
  
    zz11[i] = decay_rate11;

end


# save the Julia vectors of decay rates as npz files, so they can be imported to Python:
npzwrite("data/zz01.npz", zz01)
npzwrite("data/zz10.npz", zz10)
npzwrite("data/zz11.npz", zz11)